-- *Created by **Renish Rauniyar***

*Original Author and Credits: Michael Tsu*

# Initial Setup

In [1]:
## importing packages and setting connector to snowflake
import pandas as pd
from drg_connect import Snowflake
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', None)
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings('ignore')

In [3]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW', database = 'Project_ANALYTICS', warehouse = 'RWD_WH_ANALYTICS', schema = 'presales')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

**Pulling physician details**

** Objective ** :
        *The main aim of the exercise is to extract the physician information such as demographic info, speciality, etc. as well as its primary affiliations and parent HCO*

Input: 
    
    Table with list of physician NPI's 
    Please enter table in the field below.

Output: 

    Table with physician, primary affiliation details, secondary affiliation details and parent entity
    Enter table name below

   ***Physician Mapping***

**Step 1:**

Pull the basic physician information from rwd_db.rwd.healthbase_practitioner using the npi list such as :
1. Name
2. Address
3. Mailing or practice locations (choose one according to requirement)
4. Healthbase ID
5. Phone no
6. Physician State, city, etc

(These details are mostly extracted from https://www.hipaaspace.com/ but are subjected to change as per the algorithm set by the healthbase team)

*** Speciality is obtained in step 7

**Step 2:**

After step 1, we use rwd_db.rwd.healthbase_affiliationsadvanced to get its primary affiliation i.e. node_2_healthbase_id and its details. So, overall: 
1. Node 1 : Healthcare Physician (HCP)
2. Node 2 : Immediate Primary Affiliation

Exp: for NPI 1295978039- NILANG	PATEL
            Primary Affiliation (node_2): DF:492634	LIBERTY DIALYSIS PETERSBURG

**Step 3:**

The third step is to get parent affiliation from table: rwd_db.rwd.healthbase_parent_affiliations to find the final parent  using 'score = 100'.
1. Node 1 : Immediate Primary Affiliation (node 2 in step 2 above)
2. Node 2 : Final Parent (HCO)
    
    For the same exp:
    Exp: for NPI 1295978039- NILANG PATEL
            Primary Affiliation (node_2 in step 2): DF:492634	LIBERTY DIALYSIS PETERSBURG
            Parent (node_2 in current step): HS:C0001333141	FRESENIUS MEDICAL CARE CORP
            Source : https://www.healthcare4ppl.com/dialysis/virginia/petersburg/liberty-dialysis-petersburg-492634.html                       (external verification)

**Step 4:** 

Find all the facility NPI's (aggregated into a single field: associated with the primary affiliations) i.e. all the NPI's associated with the primary affiliation and secondary affiliation. This is done since we cannot find a single NPI associated with a particular Healthbase_id and thus we collate each NPI associated with it to a single cell.
Using: rwd_db.rwd.healthbase_entitylookup 

**Step 5:**

We can pull the demographic details for parent and primary affiliations from rwd_db.rwd.healthbase_entitysummary table. Details like:
1. Name of the parent or primary affiliation
2. Address
3. City
4. State
5. Zip
etc

Additionally, we pull and populate the details of the secondary affiliations for all the physicians. The main aim of this is to provide an alternate affiliation (high in rank) where they can find the physician. 
Note: There is an additional process of assigning the secondary affiliation using the relationship_type and distance between physician and the primary affiliation. Chapter: 2.3

**Step 6:** 

For the NPI's (both physician and facility) which are not available in the healthbase_summary table, we can get the details from the CMS table: rwd_db.rwd.third_party_cmsnpi. In addition to this, we can use facility-specific healthbase tables to get the details for those NPI's. 

   ***Physician Speciality***

**Step 7:**

Using taxonomy codes from rwd_db.rwd.healthbase_practitioner table we can determine the primary speciality of a physician (special codes for specialities: To be taken where switch value = "True"):

Map the taxonomy_code with a standard table created for code mapping to get the speciality for physicians.
rwd_db.rwd.healthbase_taxonomycode 
** Aggregated speciality is all the specialities of the physician collected in a single cell.

**QC Steps Followed:**

1. Limited the physician, primary affiliation, secondary affiliation and parent affiliation data to the 50 US states (plus Puerto Rico (PR) and District of Columbia (DC))  
2. Zip codes which are less than 5 digits have been padded with 0's from the left. 
3. Phone numbers have been formatted as per the standard US formats
4. Availability of geo_latitude and geo_longitude values for physician, primary affiliation and secondary affiliation to check the state mismatch. Solves the issue of cross verification using secondary sources.
3. For the physicians which have a valid primary affiliation (healthbase_id) but the details (name, state, zip, etc) corresponding to it is null, then populated their details using the secondary affiliation and made secondary affiliation null

# Physician Mapping

## Populating Physician Details

In [57]:
input_table = '(select distinct PHYSICIAN_NPI as npi from project_analytics.presales.oxy_all_phy)'
output_table = 'project_analytics.presales.oxy_phy_det_1'

In [58]:
%%read_sql

Create OR replace  transient TABLE testing_one_phy_row_1_ren  AS

SELECT a.*, 
---------------------------------------- Parent Details from entity_summary ----------------------------------------
       C.NAME  as Parent_Name, 
       case when c.address_line_2  is null then c.address_line_1
               when  c.address_line_2 is not null and c.address_line_1 is not null 
                  then c.address_line_1|| ', ' || c.address_line_2
       end as Parent_Address,  
       c.city as  Parent_City, 
       RIGHT(c.state,2) as Parent_State, 
       left(c.zip_code, 5)    AS Parent_Zip_code, 
---------------------------------------- Primary Affiliation Details from entity_summary ----------------------------------------   
       b.NAME  as Primary_Affiliation_Name, 
       case when b.address_line_2  is null then b.address_line_1
           when  b.address_line_2 is not null and b.address_line_1 is not null 
              then b.address_line_1|| ', ' || b.address_line_2
       end as Primary_Affiliation_Address,  
       b.city as  Primary_Affiliation_City, 
       RIGHT(b.state,2) as Primary_Affiliation_State, 
       left(b.zip_code, 5)    AS Primary_AFFILIATION_Zip_code, 
       b.GEO_LATITUDE as primary_aff_latitude,
       b.GEO_LONGITUDE as primary_aff_longitude,
       (haversine(b.GEO_LATITUDE, b.GEO_LONGITUDE, a.physician_latitude, a.physician_longitude) * 0.621371)
           as Dist_phy_primary_miles,
       CASE
            WHEN b.healthbase_id IS NOT NULL THEN 1 
                 ELSE 0 
       end AS primary_aff_in_hb , 
       Row_number() OVER (partition BY a.hcp ORDER BY a.hcp_entity_multi) AS Affiliation_Rank 
  FROM (SELECT DISTINCT f.npi,
                        e.healthbase_id, 
                        e.is_active_entity, 
---------------------------------------- Physician Details from practioner ----------------------------------------
                        e.provider_other_name_prefix_text, 
                        e.provider_first_name as Physician_First_Name, 
                        e.provider_middle_name as Physician_Middle_Name, 
                        e.provider_last_name_legal_name as Physician_Last_Name, 
                        e.provider_GENDER_CODE AS PHYSICIAN_GENDER,
                        case when e.provider_second_line_business_practice_location_address  is null 
                                    then e.provider_first_line_business_practice_location_address
                                       when e.provider_first_line_business_practice_location_address is not null 
                                         and  e.provider_second_line_business_practice_location_address is not null 
                                             then e.provider_first_line_business_practice_location_address || ', ' ||
                                                       e.provider_second_line_business_practice_location_address
                         end as Physician_Practice_Address, 
                        e.provider_business_practice_location_address_city_name as Physician_City, 
                        RIGHT(e.provider_business_practice_location_address_state_name,2)  AS Physician_State,
                        LEFT(e.provider_business_practice_location_address_postal_code,5) AS Physician_Zip_Code,
                        e.PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_TELEPHONE_NUMBER AS Physician_Phone,
                        e.GEO_LATITUDE as physician_latitude,
                        e.GEO_LONGITUDE as physician_LONGITUDE,
                        case when a.node_1_healthbase_id is not null then a.node_1_healthbase_id
                            else e.healthbase_id
                        end AS HCP, 
                        a.relationship_type, 
                        a.node_2_healthbase_id AS PRIMARY_AFFILIATION, 
                        c.npi as collated_npi, 
                        a.score1, 
                        b.node_2_healthbase_id as Entity_Parent,  
                        b.score,
                        CASE 
                          WHEN a.relationship_type = 'WORKS_AT' THEN 1 
                          WHEN a.relationship_type = 'BILLS_TO' THEN 2 
                          WHEN a.relationship_type = 'AFFILIATED' THEN 3 
                          ELSE 0 
                        END                    AS Relationship_type_multi, 
                        --Relationsip type prioritization  
                        CASE 
                          WHEN a.relationship_type is null THEN Cast(4 AS FLOAT)+( 
                          Concat('.', Lpad(100 - score1, 2, 
                                      '00')) ) 
                          WHEN a.relationship_type = 'WORKS_AT' THEN 
                          Cast(1 AS FLOAT) + ( 
                          Concat('.', Lpad(100 - score1, 2, 
                                      '00')) ) 
                          WHEN a.relationship_type = 'BILLS_TO' THEN 
                          Cast(2 AS FLOAT) + ( 
                          Concat('.', Lpad(100 - score1, 2, 
                                      '00')) ) 
                          WHEN a.relationship_type = 'AFFILIATED' THEN 
                          Cast(3 AS FLOAT) + ( 
                          Concat('.', Lpad(100 - score1, 2, 
                                      '00')) ) 
                          ELSE 0 
                        END                    AS HCP_Entity_multi ,
                        CASE 
                            WHEN e.npi IS NOT NULL THEN 1 
                            ELSE 0 
                        end AS physician_in_hb
        --HCP to Entity Affiliation Score  
                   FROM   {input_table} f
                   LEFT   JOIN (SELECT * FROM rwd_db.rwd.healthbase_practitioner
                                 WHERE upper(RIGHT(provider_business_practice_location_address_state_name,2)) in
                                             ('AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY',
                                              'LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH',
                                              'OK','OR','PA','RI','SC','SD','TN','TX','PR','UT','VA','VT','WA','WI','WV','WY')
                                )e 
                          ON f.npi = cast(e.npi as varchar)
                   left   join  rwd_db.rwd.healthbase_affiliationsadvanced a 
                          ON a.node_1_healthbase_id = e.healthbase_id   
-------------------------------- Choosing parent with "score = 100": Key to define ownership ----------------------        
                   LEFT   JOIN (select * from (select *, row_number()over(partition by node_1_healthbase_id 
                                                              order by score) as Parent_order
                                                from rwd_db.rwd.healthbase_parent_affiliations
                                               where score = 100)
                                 where parent_order = 1) b 
                          ON a.node_2_healthbase_id = b.node_1_healthbase_id 
                    -- Finding All NPI's affiliated with Primary Affiliation
                   LEFT   JOIN (SELECT healthbase_id, 
                                       Listagg(identifier, ', ') within group (order by identifier) AS NPI 
                                  FROM   rwd_db.rwd.healthbase_entitylookup 
                                  WHERE  identifier_type = 'NPI' 
                                  GROUP  BY healthbase_id
                                  ) C 
                          ON a.node_2_healthbase_id = c.healthbase_id 
                    and  e.healthbase_id LIKE 'P:%') a 
       LEFT JOIN (SELECT * FROM rwd_db.rwd.healthbase_entitysummary
                   WHERE upper(RIGHT(state,2)) in 
                         ('AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY',
                          'LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH',
                          'OK','OR','PA','RI','SC','SD','TN','TX','PR','UT','VA','VT','WA','WI','WV','WY')
                  ) b 
                            ON a.PRIMARY_AFFILIATION = b.healthbase_id
       LEFT JOIN (SELECT * FROM rwd_db.rwd.healthbase_entitysummary
                   WHERE upper(RIGHT(state,2)) in 
                         ('AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','ID','IL','IN','KS','KY',
                          'LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH',
                          'OK','OR','PA','RI','SC','SD','TN','TX','PR','UT','VA','VT','WA','WI','WV','WY')
                  ) C
                            ON a.Entity_Parent = C.healthbase_id ;

Query started at 04:51:56 PM India Standard Time; Query executed in 2.91 m

status
0  Table TESTING_ONE_PHY_ROW_1_REN successfully c...

In [59]:
%%read_sql
      select count (distinct npi), count (distinct primary_affiliation), count (distinct entity_parent),
              count (primary_affiliation), count (entity_parent)
        from testing_one_phy_row_1_ren    
        ; 

Query started at 04:54:51 PM India Standard Time; Query executed in 0.07 m

COUNT (DISTINCT NPI)  COUNT (DISTINCT PRIMARY_AFFILIATION)  \
0               1208717                                248939   

   COUNT (DISTINCT ENTITY_PARENT)  COUNT (PRIMARY_AFFILIATION)  \
0                          159159                      4864625   

   COUNT (ENTITY_PARENT)  
0                4640112

## One Row per Physician

In [60]:
%%read_sql 
Create or replace transient table TESTING_FINAL_ONE_REN as 
    select distinct a.NPI  as Physician_Npi
                   ,a.PHYSICIAN_FIRST_NAME
                   ,a.PHYSICIAN_MIDDLE_NAME
                   ,a.PHYSICIAN_LAST_NAME
                   ,a.PHYSICIAN_GENDER
                   ,a.PHYSICIAN_PRACTICE_ADDRESS
                   ,a.PHYSICIAN_CITY 
                   ,a.PHYSICIAN_STATE
                   ,a.PHYSICIAN_ZIP_CODE
                   ,case when length(A.Physician_Phone)<> 10 then a.Physician_Phone
                    else ('(' || SUBSTRING(a.Physician_Phone,1,3) || ') ' ||
                            SUBSTRING(a.Physician_Phone,4,3) || '-' || SUBSTRING(a.Physician_Phone,7,4))
                    end as Physician_Phone
                   ,a.physician_latitude
                   ,a.physician_longitude
                   ,NULL AS PHYSICIAN_MEDICAL_SCHOOL
                   ,NULL AS PHYSICIAN_GRADUATION_YEAR
                   ,NULL AS PHYSICIAN_PRACTICE_DURATION
                   ,a.IS_ACTIVE_ENTITY
                   ,a.HCP
                   ,a.PROVIDER_OTHER_NAME_PREFIX_TEXT  as Physician_Prefix      
                   ,a.COLLATED_NPI as PRIMARY_AFFILIATION_FACILITY_NPI
                   ,a.PRIMARY_AFFILIATION
                   ,a.PRIMARY_AFFILIATION_NAME
                   ,a.PRIMARY_AFFILIATION_ADDRESS
                   ,a.PRIMARY_AFFILIATION_CITY 
                   ,a.PRIMARY_AFFILIATION_STATE
                   ,a.PRIMARY_AFFILIATION_ZIP_CODE
                   ,a.primary_aff_latitude
                   ,a.primary_aff_longitude
                   ,a.Dist_phy_primary_miles
                   ,B.COLLATED_NPI as SECONDARY_AFFILIATION_FACILITY_NPI
                   ,B.PRIMARY_AFFILIATION AS SECONDARY_AFFILIATION
                   ,B.PRIMARY_AFFILIATION_NAME AS SECONDARY_AFFILIATION_NAME
                   ,B.PRIMARY_AFFILIATION_ADDRESS AS SECONDARY_AFFILIATION_ADDRESS
                   ,B.PRIMARY_AFFILIATION_CITY AS SECONDARY_AFFILIATION_CITY 
                   ,B.PRIMARY_AFFILIATION_STATE AS SECONDARY_AFFILIATION_STATE
                   ,B.PRIMARY_AFFILIATION_ZIP_CODE AS SECONDARY_AFFILIATION_ZIP_CODE
                   ,B.primary_aff_latitude as secondary_aff_latitude
                   ,B.primary_aff_longitude as secondary_aff_longitude
                   ,B.Dist_phy_primary_miles AS  Dist_phy_secondary_miles
                   ,a.ENTITY_PARENT
                   ,a.PARENT_NAME 
                   ,a.PARENT_ADDRESS
                   ,a.PARENT_CITY 
                   ,a.PARENT_STATE
                   ,a.PARENT_ZIP_CODE
                   ,a.RELATIONSHIP_TYPE
                   ,a.SCORE1
                   ,a.SCORE
                   ,a.RELATIONSHIP_TYPE_MULTI
                   ,a.HCP_ENTITY_MULTI
                   ,a.physician_in_hb
                   ,a.PRIMARY_AFF_IN_HB
                   ,NULL AS physician_in_cms
                   ,NULL AS PRIMARY_IN_CMS
               from (select * from testing_one_phy_row_1_ren where Affiliation_Rank = 1 ) A
               left join (select * from testing_one_phy_row_1_ren where Affiliation_Rank = 2 ) B
               on a.npi  = b.npi

Query started at 04:54:56 PM India Standard Time; Query executed in 0.24 m

status
0  Table TESTING_FINAL_ONE_REN successfully created.

## Possible Alternate way of finding secondary affiliation
The given table matches the state of physicians and primary affiliation and takes into account those physicians which don't have the same state or having neighbouring states.. The main aim is to give the facilities which are nearby to the physician and have a better score and least distance between the physician and facility.
Thus instead of prioritizing relationship type column first, first we pull the facility nearby to the physician and using a mixture of relationship_type and score i.e. score1 > 50
Note: Taking score1> 50 eliminates the primary affiliations where the physicians has been to a few times and has a relationship_type as "Works_at"

Eg: For a physician having multiple affiliations, and a similar distance

Scenario 1: If a physician has 
    
    Works_at facility A with score 28
    
       vs
    Bills_to facility B with score 59

Result: Take Secondary affiliation as B

Scenario 2: A Physician

    Works_at facility A with score 28
    
       vs
    Works_at facility B with score 58 
    
       vs
    Bills_to facility C with score 89
    
Result: Take Secondary affiliation as B

Scenario 3: A Physician

    Bills_to facility A with score 28
    
       vs
    Bills_to facility B with score 58 
    
       vs
    Bills_to facility C with score 89
    
Result: Take Secondary affiliation as C

In [8]:
%%read_sql
Create or replace transient table temp_table_1 as
with t1 as 
(
    select *, row_number() over(partition by npi order by Affiliation_Rank asc, Dist_phy_primary_miles asc) as rr 
      from TESTING_ONE_PHY_ROW_1_REN 
     where npi in (select distinct physician_npi from TESTING_FINAL_ONE_REN
                     where upper (nvl(physician_state,'')) <> upper(primary_affiliation_state))
       and score1 > 50
       and affiliation_rank  <> 1 
)

(select * from t1 where rr=1)

Query started at 01:00:18 PM India Standard Time; Query executed in 0.07 m

status
0  Table TEMP_TABLE_1 successfully created.

In [9]:
%%read_sql
      select count (distinct Npi), count (distinct primary_affiliation),count (primary_affiliation),
              count (distinct entity_parent),count (entity_parent)
        from temp_table_1    
        ; 

Query started at 01:00:22 PM India Standard Time; Query executed in 0.08 m

COUNT (DISTINCT NPI)  COUNT (DISTINCT PRIMARY_AFFILIATION)  \
0                   517                                   355   

   COUNT (PRIMARY_AFFILIATION)  COUNT (DISTINCT ENTITY_PARENT)  \
0                          517                             199   

   COUNT (ENTITY_PARENT)  
0                    466

## Physician Details from CMS

In [61]:
%%read_sql
BEGIN;
UPDATE TESTING_FINAL_ONE_REN   A
SET 
                A.PHYSICIAN_FIRST_NAME= b.provider_first_name   , 
                A.PHYSICIAN_LAST_NAME = b. provider_last_name_legal_name , 
                A.Physician_Practice_Address = case when b.provider_second_line_business_practice_location_address  is null 
                   then b.provider_first_line_business_practice_location_address
                      when b.provider_first_line_business_practice_location_address is not null 
                        and  b.provider_second_line_business_practice_location_address is not null 
                            then b.provider_first_line_business_practice_location_address || ', ' ||
                                      b.provider_second_line_business_practice_location_address
                end, 
                A.PHYSICIAN_CITY= b.provider_business_practice_location_address_city_name , 
                A.Physician_State = RIGHT(b.provider_business_practice_location_address_state_name,2), 
                A.Physician_Zip_Code = LEFT(b.provider_business_practice_location_address_postal_code,5),  
                A.Physician_Phone = b.PROVIDER_BUSINESS_PRACTICE_LOCATION_ADDRESS_TELEPHONE_NUMBER,
                --c.provider_first_name  as Primary_Affiliation_Name,                                                                                                  
                --c.provider_last_name_legal_name AS Primary_Affiliation_Legal_Name,                                                                                   
                --c.provider_organization_name_legal_business_name AS Primary_Affiliation_Business_Name,
                A.PHYSICIAN_IN_CMS = CASE 
                                            WHEN b.npi IS NOT NULL THEN 1 
                                            ELSE 0 
                                      end   
FROM            rwd_db.rwd.third_party_cmsnpi B
WHERE           a.PHYSICIAN_NPI = cast (b.NPI AS VARCHAR) 
AND             A.physician_in_hb = 0;
COMMIT;

Query started at 04:55:10 PM India Standard Time; Query executed in 0.05 mQuery started at 04:55:13 PM India Standard Time; Query executed in 0.10 mQuery started at 04:55:19 PM India Standard Time; Query executed in 0.05 m

status
0  Statement executed successfully.

In [62]:
%%read_sql
BEGIN;
UPDATE TESTING_FINAL_ONE_REN   A
---- GENDER DETAILS FROM CMS (due to lack of fill rate in Healthbase)
SET 
                A.PHYSICIAN_GENDER= upper(b.provider_gender_code)    
FROM            rwd_db.rwd.third_party_cmsnpi B
WHERE           a.PHYSICIAN_NPI = cast (b.NPI AS VARCHAR) 
AND             A.PHYSICIAN_GENDER IS NULL;
COMMIT;

Query started at 04:55:22 PM India Standard Time; Query executed in 0.05 mQuery started at 04:55:25 PM India Standard Time; Query executed in 0.18 mQuery started at 04:55:36 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

In [63]:
%%read_sql
----------------------------------------- UPDATING MEDICAL SCHOOL AND GRAD YEAR -----------------------------------------
BEGIN;
UPDATE TESTING_FINAL_ONE_REN   A
SET 
                A.PHYSICIAN_MEDICAL_SCHOOL = upper(b.MED_SCH),
                A.PHYSICIAN_GRADUATION_YEAR = upper(b.GRD_YR),
                A.PHYSICIAN_PRACTICE_DURATION = 2018 - B.GRD_YR  -- Business rule assuming practice after graduation
FROM            rwd_db.rwd.healthbase_physiciancompare  B
WHERE           a.PHYSICIAN_NPI = cast (b.NPI AS VARCHAR);
COMMIT;

Query started at 04:55:39 PM India Standard Time; Query executed in 0.05 mQuery started at 04:55:42 PM India Standard Time; Query executed in 0.29 mQuery started at 04:55:59 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

## Primary Affiliation Details from CMS

In [64]:
%%read_sql
BEGIN;
UPDATE  TESTING_FINAL_ONE_REN   A
SET             A.Primary_Affiliation_Name = c.PROVIDER_ORGANIZATION_NAME_LEGAL_BUSINESS_NAME,
                A.Primary_Affiliation_Address= case when c.provider_second_line_business_practice_location_address  is null 
                   then c.provider_first_line_business_practice_location_address
                      when c.provider_first_line_business_practice_location_address is not null 
                         and  c.provider_second_line_business_practice_location_address is not null 
                            then c.provider_first_line_business_practice_location_address || ' ' ||
                                      c.provider_second_line_business_practice_location_address
                end ,
                A.Primary_Affiliation_City = c.provider_business_practice_location_address_city_name , 
                A.Primary_Affiliation_State = RIGHT(c.provider_business_practice_location_address_state_name,2),
                A.Primary_AFFILIATION_Zip_Code = LEFT(c.provider_business_practice_location_address_postal_code,5),
                --c.provider_other_organization_name as Primary_Affiliation_Other_Name, 
                A.PRIMARY_IN_CMS = CASE 
                                        WHEN c.npi IS NOT NULL THEN 1 
                                        ELSE 0 
                                end
FROM            rwd_db.rwd.third_party_cmsnpi c
WHERE           split_part(a.primary_affiliation_facility_npi,',',1)  = cast(c.npi AS VARCHAR)
AND             a.Primary_Affiliation_Name is null;
COMMIT;

Query started at 04:56:03 PM India Standard Time; Query executed in 0.05 mQuery started at 04:56:06 PM India Standard Time; Query executed in 0.15 mQuery started at 04:56:15 PM India Standard Time; Query executed in 0.05 m

status
0  Statement executed successfully.

## Get Data from Facility-specific tables
For those facilities for which data was not found in healthbase_summary as well as CMS. There may be a valid healthbase id but the details corresponsding to it is not present

In [65]:
%%read_sql
update TESTING_FINAL_ONE_REN A
        set A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.PO_ST_ADR
           ,A.Primary_Affiliation_CITY = B.PO_CITY_NAME
           ,A.Primary_Affiliation_STATE = right(B.PO_STATE_CD,2)
           ,A.Primary_Affiliation_ZIP_Code = B.PO_ZIP_CD
        from 
            (select    HEALTHBASE_ID,
                       PO_FAC_NAME,
                       PO_ST_ADR,
                       PO_CITY_NAME,
                       right(PO_STATE_CD,2) as PO_STATE_CD ,
                       left(PO_ZIP_CD,5) as PO_ZIP_CD  
                 from  rwd_db.RWD.HEALTHBASE_HOSPITAL
                 where healthbase_id in (select primary_affiliation from TESTING_FINAL_ONE_REN
                                          where primary_affiliation_name is null)
             )B
     where A.primary_affiliation = B.HEALTHBASE_ID;

Query started at 04:56:18 PM India Standard Time; Query executed in 0.08 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [66]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd

    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   left(po_zip_cd,5) as po_zip_cd 
            FROM   rwd_db.rwd.healthbase_asc 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL )
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:56:23 PM India Standard Time; Query executed in 0.09 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [67]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   org_lgl_nm                               AS PO_FAC_NAME, 
                   COALESCE(address_line_1, address_line_1) AS PO_ST_ADR, 
                   city                                     AS PO_CITY_NAME, 
                   RIGHT(state, 2)                          AS PO_STATE_CD, 
                   left(zip_code,5)                         AS PO_ZIP_CD 
            FROM   rwd_db.rwd.healthbase_physiciangroup 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS  NULL )
                                      )B
    WHERE  A.primary_affiliation  = B.healthbase_id; 

Query started at 04:56:28 PM India Standard Time; Query executed in 0.17 m

number of rows updated  number of multi-joined rows updated
0                      24                                    0

In [68]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   left(po_zip_cd ,5) as po_zip_cd
            FROM   rwd_db.rwd.healthbase_federallyqualifiedhealthcenter 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                         FROM   TESTING_FINAL_ONE_REN 
                                         WHERE  primary_affiliation_name IS  NULL )
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:56:39 PM India Standard Time; Query executed in 0.08 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [69]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   organization_name              AS PO_FAC_NAME, 
                   COALESCE(address_1, address_1) AS PO_ST_ADR, 
                   city                           AS PO_CITY_NAME, 
                   RIGHT(state, 2)                AS PO_STATE_CD, 
                   left(postal_code,5)            AS PO_ZIP_CD 
            FROM   rwd_db.rwd.healthbase_rogueentity 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL )
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:56:44 PM India Standard Time; Query executed in 0.19 m

number of rows updated  number of multi-joined rows updated
0                    3925                                    0

In [70]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   org_name                                 AS PO_FAC_NAME, 
                   COALESCE(address_line_1, address_line_1) AS PO_ST_ADR, 
                   city                                     AS PO_CITY_NAME, 
                   RIGHT(state, 2)                          AS PO_STATE_CD, 
                   left(zip_code,5)                         AS PO_ZIP_CD 
            FROM   rwd_db.rwd.healthbase_businessentity 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS  NULL )
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:56:55 PM India Standard Time; Query executed in 0.10 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [71]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   left(po_zip_cd,5)  as po_zip_cd
            FROM   rwd_db.rwd.healthbase_nursinghome 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL)
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:01 PM India Standard Time; Query executed in 0.07 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [72]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   po_zip_cd 
            FROM   rwd_db.rwd.healthbase_homehealth 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL)
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:05 PM India Standard Time; Query executed in 0.07 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [73]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   po_zip_cd 
            FROM   rwd_db.rwd.healthbase_dialysisfacility 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL)
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:10 PM India Standard Time; Query executed in 0.17 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [74]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   po_zip_cd 
            FROM   rwd_db.rwd.healthbase_hospice 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL )
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:20 PM India Standard Time; Query executed in 0.07 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [75]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   po_zip_cd 
            FROM   rwd_db.rwd.healthbase_portablexray 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL)
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:25 PM India Standard Time; Query executed in 0.06 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

In [76]:
%%read_sql
UPDATE TESTING_FINAL_ONE_REN A 
    SET     A.Primary_Affiliation_Name = B.PO_FAC_NAME
           ,A.Primary_Affiliation_Address = B.po_st_adr 
           ,A.Primary_Affiliation_CITY = B.po_city_name
           ,A.Primary_Affiliation_STATE = B.po_state_cd
           ,A.Primary_Affiliation_ZIP_Code = B.po_zip_cd
    FROM   (SELECT healthbase_id, 
                   po_fac_name, 
                   po_st_adr, 
                   po_city_name, 
                   RIGHT(po_state_cd, 2) AS PO_STATE_CD, 
                   po_zip_cd 
            FROM   rwd_db.rwd.healthbase_intermediatecarefacility 
            WHERE  healthbase_id IN (SELECT primary_affiliation 
                                     FROM   TESTING_FINAL_ONE_REN 
                                     WHERE  primary_affiliation_name IS NULL)
                                      )B
    WHERE  A.primary_affiliation = B.healthbase_id; 

Query started at 04:57:28 PM India Standard Time; Query executed in 0.07 m

number of rows updated  number of multi-joined rows updated
0                       0                                    0

## Details for the primary affiliation from secondary

In [77]:
%%read_sql
begin;
update TESTING_FINAL_ONE_REN a
set a.PRIMARY_AFFILIATION_FACILITY_NPI = b.SECONDARY_AFFILIATION_FACILITY_NPI
    ,a.primary_affiliation = b.secondary_affiliation
    ,a.primary_affiliation_name = b.secondary_affiliation_name
    ,A.Primary_Affiliation_CITY = B.secondary_Affiliation_CITY
    ,A.Primary_Affiliation_STATE = B.secondary_Affiliation_STATE
    ,A.Primary_Affiliation_ZIP_Code = B.secondary_Affiliation_ZIP_Code
    from TESTING_FINAL_ONE_REN b
    where a.primary_affiliation_name is null
    and a.primary_affiliation_facility_npi not in ('1234567893','1999999992','1999999976');
commit;    

Query started at 04:57:33 PM India Standard Time; Query executed in 0.06 mQuery started at 04:57:36 PM India Standard Time; Query executed in 1.22 mQuery started at 04:58:49 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

## Set secondary blank if primary= secondary

In [78]:
%%read_sql
begin;
update TESTING_FINAL_ONE_REN a
set a.SECONDARY_AFFILIATION_FACILITY_NPI = ''
    ,a.SECONDARY_affiliation = ''
    ,a.SECONDARY_affiliation_name = ''
    ,A.SECONDARY_Affiliation_CITY = ''
    ,A.SECONDARY_Affiliation_STATE = ''
    ,A.SECONDARY_Affiliation_ZIP_Code = ''
    from TESTING_FINAL_ONE_REN b
    where a.primary_affiliation = B.SECONDARY_affiliation;
commit;

Query started at 04:58:53 PM India Standard Time; Query executed in 0.05 mQuery started at 04:58:56 PM India Standard Time

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 000604 (57014): SQL execution canceled [SQL: "update TESTING_FINAL_ONE_REN a\nset a.SECONDARY_AFFILIATION_FACILITY_NPI = ''\n    ,a.SECONDARY_affiliation = ''\n    ,a.SECONDARY_affiliation_name = ''\n    ,A.SECONDARY_Affiliation_CITY = ''\n    ,A.SECONDARY_Affiliation_STATE = ''\n    ,A.SECONDARY_Affiliation_ZIP_Code = ''\n    from TESTING_FINAL_ONE_REN b\n    where a.primary_affiliation = B.SECONDARY_affiliation;"] (Background on this error at: http://sqlalche.me/e/f405)

# Creating  Table for Physician - speciality
Creating list of All NPI with their aggregated speciality with interest of our project

## Getting Aggregated Speciality

### Pulling all taxonomy code for each NPI

In [79]:
%%read_sql
Create or replace table   stacking_speciality as 
    Select cast(B.npi as varchar)   as NPI
          ,B.spec_num
          ,B.specialty_code
     From (select npi, 1  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_1"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
             where HEALTHCARE_PROVIDER_TAXONOMY_CODE_1 is not null
           Union
           select npi, 2  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_2"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
            where HEALTHCARE_PROVIDER_TAXONOMY_CODE_2 is not null
           Union
           select npi, 3  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_3"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
           where HEALTHCARE_PROVIDER_TAXONOMY_CODE_3 is not null
           Union
           select npi, 4  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_4"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
           where HEALTHCARE_PROVIDER_TAXONOMY_CODE_4 is not null
           Union
           select npi, 5  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_5"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
           where HEALTHCARE_PROVIDER_TAXONOMY_CODE_5 is not null
           Union
           select npi, 6  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_6"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
            where HEALTHCARE_PROVIDER_TAXONOMY_CODE_6 is not null
           Union
           select npi, 7  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_7"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
             where HEALTHCARE_PROVIDER_TAXONOMY_CODE_7 is not null
             Union
           select npi, 8  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_8"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
           where HEALTHCARE_PROVIDER_TAXONOMY_CODE_8 is not null
           Union
           select npi, 9  as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_9"  as specialty_code 
             from rwd_db.rwd.healthbase_practitioner 
            where HEALTHCARE_PROVIDER_TAXONOMY_CODE_9 is not null
           Union
           select npi, 10 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_10" as specialty_code 
           from rwd_db.rwd.healthbase_practitioner 
          where HEALTHCARE_PROVIDER_TAXONOMY_CODE_10 is not null
          Union
          select npi, 11 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_11" as specialty_code 
            from rwd_db.rwd.healthbase_practitioner 
           where HEALTHCARE_PROVIDER_TAXONOMY_CODE_11 is not null
           Union
        select npi, 12 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_12" as specialty_code 
          from rwd_db.rwd.healthbase_practitioner 
         where HEALTHCARE_PROVIDER_TAXONOMY_CODE_12 is not null
           Union
        select npi, 13 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_13" as specialty_code 
          from rwd_db.rwd.healthbase_practitioner 
         where HEALTHCARE_PROVIDER_TAXONOMY_CODE_13 is not null
        Union
        select npi, 14 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_14" as specialty_code 
          from rwd_db.rwd.healthbase_practitioner 
         where HEALTHCARE_PROVIDER_TAXONOMY_CODE_14 is not null
        Union
        select npi, 15 as spec_num, "HEALTHCARE_PROVIDER_TAXONOMY_CODE_15" as specialty_code 
          from rwd_db.rwd.healthbase_practitioner 
         where HEALTHCARE_PROVIDER_TAXONOMY_CODE_15 is not null
    )     B;


Query started at 04:59:14 PM India Standard Time; Query executed in 0.18 m

status
0  Table STACKING_SPECIALITY successfully created.

### Pulling aggregated speciality

In [80]:
%%read_sql
create or replace table stacking_speciality_1 as
Select distinct
    npi,
    Grouping,
    Classification,
    Specialization,
    case when Specialization is null then 
              case when classification is null then grouping 
                   else classification end
         else specialization
         end as collapsed_specialization
from stacking_speciality a
    left join rwd_db.rwd.healthbase_taxonomycode b
    on a.specialty_code = b.code
    order by npi,collapsed_specialization;

Query started at 04:59:25 PM India Standard Time; Query executed in 0.09 m

status
0  Table STACKING_SPECIALITY_1 successfully created.

### All Specialities for physicians

In [81]:
%%read_sql
select * from final_speciality
where physician_npi = '1124295167'

Query started at 04:59:30 PM India Standard Time; Query executed in 0.07 m

Empty DataFrame
Columns: [physician_npi, agg_specialty]
Index: []

In [82]:
%%read_sql
Create or replace table  final_speciality as
    Select distinct A.physician_npi,
            LISTAGG(b.collapsed_specialization,'|') WITHIN GROUP (order by b.NPI desc) as Agg_Specialty
      from  testing_final_one_ren    A
      left join stacking_speciality_1 B
           on a.physician_npi = b.npi
           group by a.physician_npi

Query started at 04:59:34 PM India Standard Time; Query executed in 0.08 m

status
0  Table FINAL_SPECIALITY successfully created.

## Primary Speciality

In [83]:
Physician_Speciality_SWITCH_1= ','.join(['A.*',
                          "Case   when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_1) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_1\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_2) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_2\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_3) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_3\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_4) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_4\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_5) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_5\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_6) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_6\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_7) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_7\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_8) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_8\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_9) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_9\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_10) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_10\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_11) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_11\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_12) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_12\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_13) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_13\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_14) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_14\
                                  when Upper(B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_15) = 'TRUE' then B.HEALTHCARE_PROVIDER_TAXONOMY_code_15 \
                                  when B.HEALTHCARE_PROVIDER_PRIMARY_TAXONOMY_SWITCH_1 is not null then  B.HEALTHCARE_PROVIDER_TAXONOMY_code_1 \
                          end as FINAL_taxonomy"])


In [84]:
%%read_sql
create or replace transient table primary_speciality_hb as
        SELECT {Physician_Speciality_SWITCH_1}
          FROM  final_speciality A
          LEFT JOIN  rwd_db.rwd.healthbase_practitioner   B 
               ON  A.Physician_NPI= cast(B.npi as varchar())

Query started at 04:59:39 PM India Standard Time; Query executed in 0.11 m

status
0  Table PRIMARY_SPECIALITY_HB successfully created.

In [85]:
%%read_sql
create or replace transient table primary_speciality_hb_TRUE_1 as
        SELECT a.*
               ,case when c.Specialization is null 
                    then case when c.classification is null 
                              then c.grouping 
                          else c.classification end
                else c.specialization
         end as Primary_speciality
          FROM primary_speciality_hb A
          left join rwd_db.rwd.healthbase_taxonomycode    C 
                  ON A.FINAL_TAXONOMY = C.CODE 

Query started at 04:59:46 PM India Standard Time; Query executed in 0.06 m

status
0  Table PRIMARY_SPECIALITY_HB_TRUE_1 successfull...

In [86]:
%%read_sql
select * from PRIMARY_SPECIALITY_HB_TRUE_1  
where physician_npi  =  '1124374939'
---http://www.hipaaspace.com/medical_billing/coding/national_provider_identifier/codes/npi_1992967020.aspx

Query started at 04:59:50 PM India Standard Time; Query executed in 0.06 m

physician_npi                                      agg_specialty  \
0    1124374939  Cardiovascular Disease|Student in an Organized...   

  final_taxonomy      primary_speciality  
0     207RC0000X  Cardiovascular Disease

 # Creating Final Table
 This table contains all the physician and facility level (both parent and primary affiliation) information for a particular physician. There is only one row per physician table with primary, secondary affiliation. 
 Most of the QC steps have been performed in the final output itself. 
 
 Eg: Zip code are only 5 digits
     
    Telephone number are in US format
     
    Distance between physician and primary affiliation in case of state mismatch given
    
    All the output is in upper case

In [87]:
%%read_sql
create or replace transient table {output_table} as 
    select distinct A.Physician_Npi 
                   ,UPPER(B.Agg_Specialty) AS Agg_Specialty
                   ,UPPER(b.final_taxonomy) AS final_taxonomy
                   ,UPPER(B.Primary_speciality) AS Primary_speciality
                   ,UPPER(A.IS_ACTIVE_ENTITY) AS IS_ACTIVE_ENTITY
                   ,A.HCP
                   ,UPPER(A.PHYSICIAN_FIRST_NAME) AS PHYSICIAN_FIRST_NAME
                   ,UPPER(A.PHYSICIAN_LAST_NAME) AS PHYSICIAN_LAST_NAME
                   ,UPPER(A.PHYSICIAN_GENDER) AS PHYSICIAN_GENDER
                   ,UPPER(A.PHYSICIAN_PRACTICE_ADDRESS) AS PHYSICIAN_PRACTICE_ADDRESS
                   ,UPPER(A.PHYSICIAN_CITY) AS PHYSICIAN_CITY
                   ,RIGHT(A.PHYSICIAN_STATE,2) AS PHYSICIAN_STATE
                   ,case when length(LEFT(A.PHYSICIAN_ZIP_CODE,5)) = 5 then LEFT(A.PHYSICIAN_ZIP_CODE,5)
                        else LPAD (LEFT(A.PHYSICIAN_ZIP_CODE,5),5,'0') 
                    end AS PHYSICIAN_ZIP_CODE
                   ,physician_latitude
                   ,physician_longitude
                   ,case when length(Physician_Phone)<> 10 then Physician_Phone
                    else ('(' || SUBSTRING(a.Physician_Phone,1,3) || ')' || ' ' ||
                            SUBSTRING(a.Physician_Phone,4,3) || '-' || SUBSTRING(a.Physician_Phone,7,4))
                    end as Physician_Phone        
                   ,UPPER(A.PHYSICIAN_MEDICAL_SCHOOL) AS PHYSICIAN_MEDICAL_SCHOOL
                   ,A.PHYSICIAN_GRADUATION_YEAR
                   ,A.PHYSICIAN_PRACTICE_DURATION
                   ,A.PRIMARY_AFFILIATION_FACILITY_NPI 
                   ,PRIMARY_AFFILIATION
                   ,UPPER(A.PRIMARY_AFFILIATION_NAME) AS PRIMARY_AFFILIATION_NAME
                   ,UPPER(A.PRIMARY_AFFILIATION_ADDRESS) AS PRIMARY_AFFILIATION_ADDRESS
                   ,UPPER(A.PRIMARY_AFFILIATION_CITY) AS PRIMARY_AFFILIATION_CITY
                   ,RIGHT(A.PRIMARY_AFFILIATION_STATE,2) AS PRIMARY_AFFILIATION_STATE
                   ,case when length(LEFT(A.PRIMARY_AFFILIATION_ZIP_CODE,5)) = 5 then LEFT(A.PRIMARY_AFFILIATION_ZIP_CODE,5)
                        else LPAD (LEFT(A.PRIMARY_AFFILIATION_ZIP_CODE,5),5,'0') 
                    end AS PRIMARY_AFFILIATION_ZIP_CODE
                   ,a.PRIMARY_AFF_LATITUDE
                   ,a.PRIMARY_AFF_LONGITUDE
                   ,a.Dist_phy_primary_miles
                   ,A.SECONDARY_AFFILIATION_FACILITY_NPI
                   ,A.SECONDARY_AFFILIATION
                   ,UPPER(A.SECONDARY_AFFILIATION_NAME) AS SECONDARY_AFFILIATION_NAME
                   ,UPPER(A.SECONDARY_AFFILIATION_ADDRESS) AS SECONDARY_AFFILIATION_ADDRESS
                   ,UPPER(A.SECONDARY_AFFILIATION_CITY) AS SECONDARY_AFFILIATION_CITY
                   ,RIGHT(A.SECONDARY_AFFILIATION_STATE,2) AS SECONDARY_AFFILIATION_STATE
                   ,case when length(LEFT(A.SECONDARY_AFFILIATION_ZIP_CODE,5)) = 5 then LEFT(A.SECONDARY_AFFILIATION_ZIP_CODE,5)
                         else LPAD (LEFT(A.SECONDARY_AFFILIATION_ZIP_CODE,5),5,'0') 
                     end AS SECONDARY_AFFILIATION_ZIP_CODE
                   ,a.Dist_phy_SECONDARY_MILES
                   ,UPPER(A.ENTITY_PARENT) AS ENTITY_PARENT
                   ,UPPER(A.PARENT_NAME) AS PARENT_NAME
                   ,UPPER(A.Parent_Address) AS Parent_Address
                   ,UPPER(A.PARENT_CITY) AS PARENT_CITY
                   ,RIGHT(A.PARENT_STATE,2) AS PARENT_STATE
                   ,case when length(LEFT(A.PARENT_ZIP_CODE,5)) = 5 then LEFT(A.PARENT_ZIP_CODE,5)
                        else LPAD (LEFT(A.PARENT_ZIP_CODE,5),5,'0') 
                    end AS PARENT_ZIP_CODE
              from TESTING_FINAL_ONE_REN   A
              LEFT JOIN primary_speciality_hb_TRUE_1 B
                   ON cast(A.PHYSICIAN_NPI as varchar()) = cast(B.PHYSICIAN_NPI as varchar())

Query started at 04:59:53 PM India Standard Time; Query executed in 0.25 m

status
0  Table OXY_PHY_DET_1 successfully created.

In [56]:
%%read_sql
select distinct primary_speciality from oxy_phy_det 

Query started at 01:30:40 PM India Standard Time; Query executed in 0.06 m

primary_speciality
0     ADVANCED HEART FAILURE AND TRANSPLANT CARDIOLOGY
1                                  PSYCH/MENTAL HEALTH
2                                SURGICAL TECHNOLOGIST
3                          OXYGEN EQUIPMENT & SUPPLIES
4                                         PRIMARY CARE
5                                HEMATOLOGY & ONCOLOGY
6                                    RETINA SPECIALIST
7                                  PHYSICIAN ASSISTANT
8                                         RHEUMATOLOGY
9                                      FAMILY MEDICINE
10                                   INTERNAL MEDICINE
11                                          HEPATOLOGY
12                                      OTOLARYNGOLOGY
13                                             UROLOGY
14                                 ORTHOPAEDIC SURGERY
15                                         OPTOMETRIST
16                                      WOMEN'S HEALTH
17                                        RURAL HEALTH
18                                   MILITARY HOSPITAL
19                         PROSTHETICS CASE MANAGEMENT
20                                                None
21                                  AEROSPACE MEDICINE
22                                          HEMATOLOGY
23                                           NEUROLOGY
24   STUDENT IN AN ORGANIZED HEALTH CARE EDUCATION/...
25                                             MIDWIFE
26                            CLINICAL GENETICS (M.D.)
27                 VASCULAR & INTERVENTIONAL RADIOLOGY
28                                 OTOLARYNGIC ALLERGY
29                                      LEGAL MEDICINE
30                                          NATUROPATH
31                                  PHYSICAL THERAPIST
32                                  EMERGENCY MEDICINE
33                                GYNECOLOGIC ONCOLOGY
34                                         DERMATOLOGY
35                                 OCCUPATIONAL HEALTH
36                                         HOME HEALTH
37                                       OPHTHALMOLOGY
38                         GENERAL ACUTE CARE HOSPITAL
39                    UNDERSEA AND HYPERBARIC MEDICINE
40                                  NURSE PRACTITIONER
41                                      ANESTHESIOLOGY
42                                    GASTROENTEROLOGY
43                               OCCUPATIONAL MEDICINE
44                            LICENSED PRACTICAL NURSE
45                                         GERONTOLOGY
46                                       CLINIC/CENTER
47                                       PAIN MEDICINE
48                                  TRANSPLANT SURGERY
49                                              FAMILY
50                                    GENERAL PRACTICE
51                                    MEDICAL ONCOLOGY
52                                    MEDICAL-SURGICAL
53                                      ADULT MEDICINE
54                              CRITICAL CARE MEDICINE
55                              OCCUPATIONAL THERAPIST
56                   RESPIRATORY THERAPIST, REGISTERED
57                                       FAMILY HEALTH
58                          EMERGENCY MEDICAL SERVICES
59                                          PEDIATRICS
60                  PHYSICAL MEDICINE & REHABILITATION
61                                          NEPHROLOGY
62                                         AUDIOLOGIST
63                           INTERVENTIONAL CARDIOLOGY
64                                        CHIROPRACTOR
65                                  GERIATRIC MEDICINE
66                                          PHARMACIST
67                                        FOOT SURGERY
68                                    VASCULAR SURGERY
69                                NEUROLOGICAL SURGERY
70                                     PLASTIC SURGERY
71                                    REGISTERED NURSE
72                   

In [90]:
%%read_sql
select * from oxy_phy_det_1
 where physician_npi = '1235106055'

Query started at 06:23:58 PM India Standard Time; Query executed in 0.07 m

physician_npi                                      agg_specialty  \
0    1235106055  MOHS-MICROGRAPHIC SURGERY|PROCEDURAL DERMATOLO...   

  final_taxonomy primary_speciality is_active_entity             hcp  \
0           None               None             TRUE  P:OVW1KP804V1J   

  physician_first_name physician_last_name physician_gender  \
0                  LEE               ALLEN                M   

  physician_practice_address physician_city physician_state  \
0          1455 UNION AVENUE        MEMPHIS              TN   

  physician_zip_code  physician_latitude  physician_longitude physician_phone  \
0              38104                  35                  -90  (901) 726-6655   

  physician_medical_school physician_graduation_year  \
0                     None                      None   

  physician_practice_duration primary_affiliation_facility_npi  \
0                        None                             None   

  primary_affiliation primary_affiliation_name primary_affiliation_address  \
0                None                     None                        None   

  primary_affiliation_city primary_affiliation_state  \
0                     None                      None   

  primary_affiliation_zip_code primary_aff_latitude primary_aff_longitude  \
0                         None                 None                  None   

  dist_phy_primary_miles secondary_affiliation_facility_npi  \
0                   None                               None   

  secondary_affiliation secondary_affiliation_name  \
0                  None                       None   

  secondary_affiliation_address secondary_affiliation_city  \
0                          None                       None   

  secondary_affiliation_state secondary_affiliation_zip_code  \
0                        None                           None   

  dist_phy_secondary_miles entity_parent parent_name parent_address  \
0                     None          None        None           None   

  parent_city parent_state parent_zip_code  
0        None         None            None

In [ ]:
%%read_sql
desc table {output_table}

In [ ]:
%%read_sql
      select count (distinct Physician_Npi), count (distinct PRIMARY_AFFILIATION),count (PRIMARY_AFFILIATION),
             count (distinct entity_parent),
             count (entity_parent)
        from {output_table}     
        ; --total counts: claim: 2390651	patient: 121503

In [ ]:
%%read_sql -d output_table
select * from {output_table}

In [ ]:
output_table.nunique()
output_table.to_excel('output_table.xlsx',index=False)